# ProbTTR Bayes classifier

In [1]:
from probttrtypes import Type,VarType,PRound
from utils import show

## Random variables in TTR

In ... Staffan Larsson has introduced a formulation of random variables as a TTR type.  In probabilistic `pyttr` we  create such a type as an instance of the class `VarType` (variable type).  We create a variable type by `V = VarType(Ts)` where `Ts` is a list of types.  The idea is that an object will be of type `V` with either probability 1 or 0.  For any witness for `V`, we will obtain a distribution of probalities over the types in `Ts`, that is, the sum of the probabilities that the witness is of type `T` for all the types in `Ts` will be 1.  Below we define an abstract example.

In [2]:
A1 = Type()
A2 = Type()
A3 = Type()
A1.judge('a',.3)
A2.judge('a',.3)
A3.judge('a',.4)
V1 = VarType([A1,A2,A3])
print(show(V1.query('a')))

1.0


In order to query the probability that an object is of type `T` with respect to a variable type `V`, we use the  method `query_v` as in `query_v('a',T)`.

In [3]:
for T in [A1,A2,A3]:
    print(show(V1.query_v('a',T)))

0.3
0.3
0.4


In the example above the result returned by `V1.query_v('a',T)` for any of the types on which `V1` is defined is the same as the result returned by `T.query('a')` because we declared the probabilities to form a distribution.  In the theoretical presentation of random variables in TTR it is required that the probabilities computed independently of the variable type are the same as the probabilities computed with respect to the variable type.  However, in `pyttr` we allow the variable to distribute arbitrary probability mass so that a distribution is formed.  The effect of this intuitively is to change the probability of something being a witness type  depending on what other types you are currently comparing it with.  This might be one way of dealing with examples like the difference in redness of faces, wine, coats, apples etc.

In the example below we consider an example of an apple which as an object may be considered to be greenish yellow.  However, in a classification of apples as being either green or red it would be considered to be definitely green.

In [4]:
A1.judge('b',0)
A2.judge('b',0)
A3.judge('b',0)
print(show(V1.query('b')))

0.0


In [7]:
A1.judge('c',.1)
print(show(PRound(V1.query('c'))))

1.0


In [6]:
print(show(V1.query('d')))

<=1.0
